In [1]:

### import libraries
from garminconnect import (
    Garmin,
    GarminConnectConnectionError,
    GarminConnectTooManyRequestsError,
    GarminConnectAuthenticationError,
)
from datetime import date
import pandas as pd
pd.options.display.max_columns = None
import json 
import pymongo
from datetime import datetime
from configparser import ConfigParser
pd.options.mode.chained_assignment = None

"""
Setup Mongo Connection
"""
mongoclient = "mongodb://mongo-app:27017/"
mongodb = "activities"
mongocol = "general_activities"


"""
Read config.ini file
"""
config_object = ConfigParser()
config_object.read("config.ini")

"""
Get the password
"""
userinfo = config_object["USERINFO"]


"""
Initialize Garmin client with credentials
"""
try:
    client = Garmin(userinfo["email"],userinfo["password"])
except (
    GarminConnectConnectionError,
    GarminConnectAuthenticationError,
    GarminConnectTooManyRequestsError,
) as err:
    print("Error occured during Garmin Connect Client init: %s" % err)
    quit()
except Exception:  # pylint: disable=broad-except
    print("Unknown error occured during Garmin Connect Client init")
    quit()


"""
Login to Garmin Connect portal
"""
try:
    client.login()
except (
    GarminConnectConnectionError,
    GarminConnectAuthenticationError,
    GarminConnectTooManyRequestsError,
) as err:
    print("Error occured during Garmin Connect Client login: %s" % err)
    quit()
except Exception:  # pylint: disable=broad-except
    print("Unknown error occured during Garmin Connect Client login")
    quit()


"""
Get full name from profile
"""
try:
    print(client.get_full_name())
except (
    GarminConnectConnectionError,
    GarminConnectAuthenticationError,
    GarminConnectTooManyRequestsError,
) as err:
    print("Error occured during Garmin Connect Client get full name: %s" % err)
    quit()
except Exception:  # pylint: disable=broad-except
    print("Unknown error occured during Garmin Connect Client get full name")
    quit()

"""
Get activities data
"""
try:
    activities = client.get_activities(0,10000) # 0=start, 1=limit
except (
    GarminConnectConnectionError,
    GarminConnectAuthenticationError,
    GarminConnectTooManyRequestsError,
) as err:
    print("Error occured during Garmin Connect Client get activities: %s" % err)
    quit()
except Exception:  # pylint: disable=broad-except
    print("Unknown error occured during Garmin Connect Client get activities")
    quit()

"""
Connect to Mongo DB
"""
myclient = pymongo.MongoClient(mongoclient)
mydb = myclient[mongodb]
mycol = mydb[mongocol]


#Create pandas dataframe from data
df_act = pd.DataFrame(activities)

"""
clean dataframe
"""
# select columns
full_activities = df_act[['activityName','activityType','duration','averageHR','startTimeLocal']]
# Get duration in minutes
full_activities.loc[:, 'duration_min'] = full_activities['duration']/60
# Get activity type
full_activities['activityType'] = full_activities['activityType'].apply(
    lambda x : x['typeKey'])
# convert datetime to datetime format
full_activities['startTimeLocal'] = full_activities['startTimeLocal'].apply(
    lambda x :datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
# Get week number
full_activities['week'] = full_activities['startTimeLocal'].apply(lambda x : x.strftime("%V"))
full_activities['date'] = full_activities['startTimeLocal'].apply(lambda x : x.date())

"""
Insert data into mongodb
"""
# Convert dataframe to json
records = json.loads(full_activities.T.to_json()).values()

for doc in records:
    mycol.replace_one(
        {'activityId' : doc['activityId']},
        doc,
        upsert=True
    )

Adrien


KeyError: 'activityId'

In [2]:
records

dict_values([{'activityName': 'Paris Course', 'activityType': 'running', 'duration': 3765.8068847656, 'averageHR': 148.0, 'startTimeLocal': 1594204385000, 'duration_min': 62.7634480794, 'week': '28', 'date': 1594166400000}, {'activityName': 'Paris Cyclisme', 'activityType': 'cycling', 'duration': 8601.171875, 'averageHR': 122.0, 'startTimeLocal': 1594145920000, 'duration_min': 143.3528645833, 'week': '28', 'date': 1594080000000}, {'activityName': 'Paris Course', 'activityType': 'running', 'duration': 1919.7509765625, 'averageHR': 150.0, 'startTimeLocal': 1594127095000, 'duration_min': 31.9958496094, 'week': '28', 'date': 1594080000000}, {'activityName': 'Gif-sur-Yvette Cyclisme', 'activityType': 'cycling', 'duration': 16207.3193359375, 'averageHR': 133.0, 'startTimeLocal': 1593956243000, 'duration_min': 270.1219889323, 'week': '27', 'date': 1593907200000}, {'activityName': 'Paris Course', 'activityType': 'running', 'duration': 4461.990234375, 'averageHR': 160.0, 'startTimeLocal': 15938